In [1]:
import yaml

with open("few_shot_prompts.yaml", "r") as yaml_file:
    loaded_yaml = yaml.safe_load(yaml_file)

ParserError: while parsing a block mapping
  in "few_shot_prompts.yaml", line 1, column 1
expected <block end>, but found '<scalar>'
  in "few_shot_prompts.yaml", line 89, column 82

In [2]:
for key, value in loaded_yaml.items():
    print("Question: ", key)
    print("Code: ", value)
    break

Question:  List all "My Brands" or what brands I own
Code:   # Filter for rows where MyBrand is 1
df_my_brands = data[data['MyBrand'] == 1]['Brand'].unique()
df_my_brands 


In [3]:
import pandas as pd 

data = pd.read_csv("summary.csv")

In [4]:
count = 6
ind = 0
for key, value in loaded_yaml.items():
    ind+=1
    if ind==count:
        print("Question: ", key)
        print("Code: ", value)
        exec(value)
        display(eval(value.split('\n')[-1]))
    else: 
        pass

Question:  Which brand has the highest score in the Buzz component?
Code:   # Filter the dataframe for the 'Buzz' component
buzz_df = data[data['Brand Pulse Component'] == 'Buzz']
# Ensure there are rows after filtering
assert not buzz_df.empty, "No data available for the 'Buzz' component."
# Find all the brands with the highest Buzz score
highest_buzz_rows = buzz_df.loc[buzz_df['Score']==buzz_df['Score'].max()]
highest_buzz_rows 


,Category,MyBrand,Brand,Prestige,Brand Pulse Component,Detailed Metric,Month_Start_Date,Year_Month,Year,Score,Score_Type,Channel,TotalConversations
1062,Skin Care US,1,Aveeno,NaN,Buzz,Share_Scaled,2022-08-01,2022_08 - August,2022,100.0,Sub Score,Ratings,122067
1093,Skin Care US,0,Cerave,NaN,Buzz,Share_Scaled,2022-08-01,2022_08 - August,2022,100.0,Sub Score,Social,24810
1146,Skin Care US,0,Cerave,NaN,Buzz,Share_Scaled,2022-08-01,2022_08 - August,2022,100.0,Sub Score,Search,24810
2945,Skin Care US,1,Aveeno,NaN,Buzz,Share_Scaled,2022-09-01,2022_09 - September,2022,100.0,Sub Score,Ratings,116962
2975,Skin Care US,0,Cerave,NaN,Buzz,Share_Scaled,2022-09-01,2022_09 - September,2022,100.0,Sub Score,Social,21239
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40548,Skin Care US,1,Neutrogena,NaN,Buzz,Share_Scaled,2024-05-01,2024_05 - May,2024,100.0,Sub Score,Ratings,32022
40622,Skin Care US,0,La Roche-Posay,NaN,Buzz,Share_Scaled,2024-05-01,2024_05 - May,2024,100.0,Sub Score,Search,14497
42423,Skin Care US,0,Cerave,NaN,Buzz,Share_Scaled,2024-06-01,2024_06 - June,2024,100.0,Sub Score,Social,27552
42429,Skin Care US,1,Neutrogena,NaN,Buzz,Share_Scaled,2024-06-01,2024_06 - June,2024,100.0,Sub Score,Ratings,23343


In [5]:
data

,Category,MyBrand,Brand,Prestige,Brand Pulse Component,Detailed Metric,Month_Start_Date,Year_Month,Year,Score,Score_Type,Channel,TotalConversations
0,Skin Care US,0,Shea Moisture,NaN,Action,Beat Score,2022-08-01,2022_08 - August,2022,64.162,Beat Score,Mixed,243
1,Skin Care US,0,Hawaiian Tropic,NaN,Action,Beat Score,2022-08-01,2022_08 - August,2022,35.558,Beat Score,Mixed,20
2,Skin Care US,0,La Roche-Posay,NaN,Action,Beat Score,2022-08-01,2022_08 - August,2022,64.105,Beat Score,Mixed,507
3,Skin Care US,0,Garnier,NaN,Action,Beat Score,2022-08-01,2022_08 - August,2022,70.506,Beat Score,Mixed,233
4,Skin Care US,0,Gold Bond,NaN,Action,Beat Score,2022-08-01,2022_08 - August,2022,57.731,Beat Score,Mixed,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43235,Skin Care US,0,StriVectin,NaN,Trust,Social_NPS,2024-06-01,2024_06 - June,2024,89.189,Sub Score,Social,49
43236,Skin Care US,0,Vichy,NaN,Trust,Social_NPS,2024-06-01,2024_06 - June,2024,77.344,Sub Score,Social,55
43237,Skin Care US,0,Murad,NaN,Trust,Social_NPS,2024-06-01,2024_06 - June,2024,79.048,Sub Score,Social,27
43238,Skin Care US,1,Neostrata,NaN,Trust,Social_NPS,2024-06-01,2024_06 - June,2024,72.727,Sub Score,Social,8
